In [ ]:
import requests
from multiprocessing import Pool
import pandas as pd
from lxml import etree
import re
import time
import os
import threading
class crawl:
    def __init__(self,city_name,state):
        self.state = state
        self.city_name = city_name
        self.flag = 1
        self.time_hour = ''
        self.headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
    def get_day(self):
        for year in range(2015,2020):
            if year < 2019:
                for month in range(1,13):
                    month = "%02d" % month
                    data_time = str(year) + str(month)
                    url = 'http://www.tianqihoubao.com/aqi/{0}-{1}.html'.format(self.city_name,data_time)
                    data_pd = pd.read_html(url, header=0, encoding='gbk',index_col='日期')[0] 
                    print(data_pd)
                    self.save_data(data_pd)
                time.sleep(1)
            else:
                for month in range(1,5):
                    month = "%02d" % month
                    data_time = str(year) + str(month)
                    url = 'http://www.tianqihoubao.com/aqi/{0}-{1}.html'.format(self.city_name,data_time)
                    data_pd = pd.read_html(url, header=0, encoding='gbk',index_col='日期')[0] 
                    
                    print(data_pd)
                    self.save_data(data_pd)
                time.sleep(1)
    def get_per_hour(self):
        headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
        url = 'http://www.tianqihoubao.com/aqi/{0}.html'.format(self.city_name)
        html = requests.get(url,headers=headers)
        content = html.text
        selector = etree.HTML(content)
        time_ = selector.xpath('//div[@class="desc"]/text()')[0]
        time_ =re.findall(r'数据更新时间:(.+)\(',time_)[0].strip()
        if os.path.exists(self.file_name):
            print('文件已存在，检测是否更新')
            befor_data = pd.read_csv(self.file_name,encoding='utf_8_sig')
            last = len(befor_data)
            last_time = befor_data.loc[last-1,'time']
            if last_time !=time_:
                data =pd.read_html(url, header=0, encoding='gbk')[0] 
                data['time'] = time_
                data['city'] = self.city_name
                data=data.reindex(columns=[ 'time','city','监测点', 'AQI指数', '空气质量状况', 'PM10', 'Co', 'No2', 'So2', 'O3'])
                data = data.set_index('time')
                self.save_data(data)
                self.time_hour = time_
                print('已更新')
            else:
                print('未更新')
        else:
            print('第一次创建文件')
            data =pd.read_html(url, header=0, encoding='gbk')[0] 
            data['time'] = time_
            data['city'] = self.city_name
            data=data.reindex(columns=[ 'time','city','监测点', 'AQI指数', '空气质量状况', 'PM10', 'Co', 'No2', 'So2', 'O3'])
            data = data.set_index('time')
            self.save_data(data)
            self.time_hour = time_
            
    def save_data(self,data):
        print('保存成功')
        attribute = list(data.columns)
        if self.flag ==1:
            data.to_csv(self.file_name, header=attribute,encoding='utf_8_sig',mode='a')
            self.flag = 0
        else:
            data.to_csv(self.file_name, header=None,encoding='utf_8_sig',mode='a')
    def start_crawl(self): 
        if self.state ==1:
            self.file_name = '空气质量-{0}_day.csv'.format(self.city_name)
            self.get_day()
        else:
            self.file_name = '空气质量-{0}_hour.csv'.format(self.city_name)
            self.get_per_hour()
if __name__ == '__main__':
    city_name = ['taian','qinhuangdao','beijing','jinan']
    for i in city_name:
        a = crawl(i,state=1)
        a.start_crawl()
            

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2015-01-01     良     87      224     54   119   72   54  1.60  25
2015-01-02  轻度污染    125      521     87   175   96   65  2.04  20
2015-01-03  中度污染    160      280    122   212   86   75  2.04  22
2015-01-04  重度污染    212      280    164   290  129   89  2.71  30
2015-01-05  重度污染    241      318    192   294  100   78  2.60  17
2015-01-06     良     73      185     44    90   42   43  1.32  20
2015-01-07  轻度污染    109      290     78   137   78   60  2.05  10
2015-01-08  重度污染    205      346    154   271  117   82  3.01   8
2015-01-09  严重污染    328      364    256   432  130  107  3.43   5
2015-01-10  严重污染    289      357    239   332  125  104  3.43   8
2015-01-11  中度污染    170      296    128   198   61   64  2.51   9
2015-01-12     良     72      147     49    82   58   40  1.38  15
2015-01-13     良     80      170     57    89   60   38  1.05  15
2015-01-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co   O3
日期                                                                
2015-05-01     良     94      282     58   102   33   28  0.95  119
2015-05-02     良     90      342     64   107   17   31  1.15   62
2015-05-03     良     98      307     67   127   27   31  1.22   76
2015-05-04     良     78      247     23   104   33   23  0.69   81
2015-05-05     良     89      266     54   116   34   26  0.84  107
2015-05-06     良     98      264     69   119   42   39  1.25   67
2015-05-07     良     75      155     39    84   44   27  0.70   91
2015-05-08     良     70      216     49    82   29   31  0.71   94
2015-05-09  轻度污染    105      334     77   112   28   36  1.07   50
2015-05-10     良     66      194     39    77   26   27  0.65   66
2015-05-11     优     46      103     25    47   13   35  0.71   43
2015-05-12     良     80      304     27   108   29   33  0.78   72
2015-05-13     良     91      292     53   128   31   30  0.59 

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2015-09-01     良     53      175     28    52   17   22  1.06  70
2015-09-02     优     50      160     20    46   16   30  0.86  67
2015-09-03     良     69      273     35    80   31   43  1.02  75
2015-09-04     良     84      323     57   108   47   34  1.30  83
2015-09-05     良     64      315     38    74   22   24  1.11  82
2015-09-06     良     63      289     33    66   21   22  0.86  83
2015-09-07     良     67      274     26    64   31   36  0.88  94
2015-09-08     良     72      268     43    88   42   36  1.19  89
2015-09-09     良     70      278     40    88   55   40  0.97  41
2015-09-10     良     98      346     71   127   32   28  1.44  67
2015-09-11     优     50      186     32    49   17   32  1.26  22
2015-09-12     优     43      138     20    44   13   29  0.74  36
2015-09-13     良     71      297     33    90   40   49  0.86  38
2015-09-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2016-01-01  中度污染    172      293    129   187   50   53  1.59  42
2016-01-02  严重污染    267      339    213   306   73   78  2.78  23
2016-01-03  严重污染    405      353    345   485   79  106  4.06   8
2016-01-04  严重污染    258      338    210   305   47   61  3.14  19
2016-01-05  轻度污染    116      284     86   128   44   40  1.71  18
2016-01-06  轻度污染    138      301    105   156   40   51  1.85  12
2016-01-07  轻度污染    109      269     81   124   39   58  1.72  12
2016-01-08  轻度污染    136      311     96   173   47   67  2.27  13
2016-01-09  轻度污染    145      260    108   191   61   67  2.75  16
2016-01-10  重度污染    232      328    186   293  103   79  3.06  19
2016-01-11     良     95      269     70   102   46   35  0.95  40
2016-01-12     良     87      267     63   105   42   55  1.26  23
2016-01-13  轻度污染    124      288     93   158   56   63  1.41  22
2016-01-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co   O3
日期                                                                
2016-05-01     良     89      243     46   109   17   27  0.66  122
2016-05-02     良     80      275     40   106   16   23  0.52  106
2016-05-03     良     77      299     35   101    9   22  0.34   81
2016-05-04     良     97      325     37   118   37   44  0.59  112
2016-05-05     良     85      267     32    91   16   23  0.69  132
2016-05-06  中度污染    165      337     59   239   10   25  0.52   89
2016-05-07  轻度污染    119      314     54   186   52   40  0.86   68
2016-05-08     良    100      350     59   148   36   57  1.08   61
2016-05-09     良     93      345     52   133   17   42  1.12   49
2016-05-10     良     95      317     61   129   20   46  1.07   88
2016-05-11  轻度污染    122      317     68   154   21   41  0.79  134
2016-05-12     良     66      108     29    70   17   21  0.55   92
2016-05-13     良     66      189     32    74   40   30  0.74 

...

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2018-10-01     优     31       36      7    19    3   17  0.30  59
2018-10-02     优     32       26     10    27    2   31  0.40  45
2018-10-03     优     37       41     11    32    3   38  0.44  44
2018-10-04     良     67      200     41    70    7   51  0.96  69
2018-10-05  轻度污染    105      346     78    94    7   52  1.20  66
2018-10-06     优     33       24      9    22    2   17  0.30  54
2018-10-07     优     35       38      9    34    2   35  0.34  34
2018-10-08     优     39       68      9    34    2   33  0.36  42
2018-10-09     优     23       12      3    14    2   12  0.22  43
2018-10-10     优     25       23      5    22    2   23  0.30  31
2018-10-11     优     32       45      9    29    2   32  0.32  33
2018-10-12     良     63      226     33    71    3   54  0.56  33
2018-10-13  轻度污染    107      355     80   103    6   69  0.97  44
2018-10-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2019-02-01     良     67      152     40    81    6   45  0.89  27
2019-02-02  轻度污染    138      333    103   134    7   53  1.39  17
2019-02-03  轻度污染    115      271     43   159    2   14  0.45  57
2019-02-04     良     77      172     41    99    5   24  0.80  43
2019-02-05  中度污染    153      253     97   221    9   25  0.88  35
2019-02-06     良     83      196     55   110   13   22  1.12  39
2019-02-07     优     35       50      7    28    2    8  0.32  61
2019-02-08     优     43      181     24    44    5   18  0.45  52
2019-02-09     优     31       56      8    19    3   11  0.33  56
2019-02-10     优     39      116     24    34    5   21  0.50  41
2019-02-11     良     55      163     36    51    7   28  0.72  39
2019-02-12     优     49      132     28    51    6   23  0.75  39
2019-02-13     优     39       77     21    31    2   21  0.54  44
2019-02-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co  O3
日期                                                               
2015-02-01  轻度污染    107      257     77   149   89   72  1.68  19
2015-02-02  中度污染    163      314    123   204  144   89  2.07  13
2015-02-03  重度污染    239      340    188   292  131   94  2.45  16
2015-02-04  轻度污染    150      266    104   178   66   54  1.52  30
2015-02-05     良     60       98     34    72   52   50  1.06  30
2015-02-06     良     90      175     62   113   82   57  1.13  37
2015-02-07  轻度污染    141      293     90   194   77   64  1.58  27
2015-02-08     良     68      144     33    84   56   49  1.28  28
2015-02-09     良     85      195     50   118   56   45  1.05  44
2015-02-10  轻度污染    102      194     73   140   60   43  1.11  63
2015-02-11  中度污染    200      348    155   262  130   96  2.39  26
2015-02-12  中度污染    157      280    116   195  109   79  2.26  23
2015-02-13  中度污染    167      301    126   217   99   73  2.11  39
2015-02-14

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co   O3
日期                                                                
2015-06-01  轻度污染    107      286     80   146   20   21  1.12  128
2015-06-02  轻度污染    121      342     83   162   31   31  1.18  115
2015-06-03     良     99      351     61   136   42   55  1.49   90
2015-06-04     良     87      319     40   123   24   29  0.75  105
2015-06-05     良     95      326     43   137   23   24  0.62  111
2015-06-06  轻度污染    143      360     60   233   22   21  0.56  121
2015-06-07  轻度污染    143      356     90   207   51   42  1.48  142
2015-06-08     良     95      332     58    92   19   35  1.05   96
2015-06-09     良     94      286     49   112   29   35  0.82  152
2015-06-10  轻度污染    137      342     90   190   20   21  0.99  162
2015-06-11     良     73      245     33    90   19   26  0.62   87
2015-06-12     良     89      300     39   120   32   32  0.71  115
2015-06-13     良     75      272     33    92   27   31  0.78 

            质量等级  AQI指数  当天AQI排名  PM2.5  PM10  So2  No2    Co   O3
日期                                                                
2015-10-01     良     69      251     32    82   15   37  0.69   47
2015-10-02  轻度污染    107      329     55   162   25   36  0.59   86
2015-10-03  中度污染    189      358    133   278   46   75  1.37   76
2015-10-04  重度污染    229      359    174   319   37   71  1.48   94
2015-10-05  轻度污染    118      326     86   165   24   32  0.72  118
2015-10-06  轻度污染    106      294     79   135   24   44  0.84  102
2015-10-07  轻度污染    103      323     75   119   20   49  0.86   44
2015-10-08     良     75      322     34    97   16   41  0.66   30
2015-10-09     良     87      331     42   122   29   40  0.76   52
2015-10-10     良     63      226     21    77   17   33  0.63   42
2015-10-11     良     93      332     53   127   33   57  0.84   36
2015-10-12     良     79      251     39   109   22   59  0.97   40
2015-10-13     良     94      258     58   130   35   51  0.94 

In [17]:
import requests
import
import pandas as pd
from lxml import etree
import time
from multiprocessing import Pool
import threading
class crawl:
    def __init__(self,city_name,state=1):
        self.state = state
        self.file_name = '空气质量-{0}.csv'.format(city_name)
        self.city_name = city_name
        self.flag = 1
        self.crawl_list = []
        self.headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
    def get_day(self):
        for year in range(2015,2017):
            if year < 2019:
                for month in range(1,13):
                    month = "%02d" % month
                    time = str(year) + str(month)
                    url = 'http://www.tianqihoubao.com/aqi/{0}-{1}.html'.format(self.city_name,time)
                    self.crawl_list.append(url)
                    #data_pd = pd.read_html(url, header=0, encoding='gbk',index_col='日期')[0] 
                    print('读取url:',url)
                    #self.save_data(data_pd)
            else:
                for month in range(1,5):
                    month = "%02d" % month
                    time = str(year) + str(month)
                    url = 'http://www.tianqihoubao.com/aqi/{0}-{1}.html'.format(self.city_name,time)
                    self.crawl_list.append(url)
                    #data_pd = pd.read_html(url, header=0, encoding='gbk',index_col='日期')[0] 
                    print('读取url:',url)  
    
    def crawl_save(self,url):
        print('正在爬取 ：',url)
        data_pd = pd.read_html(url, header=0, encoding='gbk',index_col='日期')[0]            
        print(data_pd)
        attribute = list(data.columns)
        if self.flag ==1:
            data.to_csv(self.file_name, header=attribute,encoding='utf_8_sig',mode='a')
            print('保存成功')
            self.flag = 0
        else:
            data.to_csv(self.file_name, header=None,encoding='utf_8_sig',mode='a')
            print('保存成功')
    def get_per_hour(self):
        headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
        url = 'http://www.tianqihoubao.com/aqi/{0}.html'.format(self.city_name)
        html = requests.get(url,headers=headers)
        content = html.text
        selector = etree.HTML(content)
        time_ = selector.xpath('//div[@class="desc"]/text()')[0]
        time_ =re.findall(r'数据更新时间:(.+)\(',time_)[0].strip()
        data =pd.read_html(url, header=0, encoding='gbk')[0] 
        data['time'] = time_
        data.reindex(columns=[ 'time','监测点', 'AQI指数', '空气质量状况', 'PM10', 'Co', 'No2', 'So2', 'O3'])
        data.set_index('time')
    def save_data(self,data):
        print('保存成功')
        attribute = list(data.columns)
        if self.flag ==1:
            data.to_csv(self.file_name, header=attribute,encoding='utf_8_sig',mode='a')
            self.flag = 0
        else:
            data.to_csv(self.file_name, header=None,encoding='utf_8_sig',mode='a')
    def start_crawl(self): 
        if self.state ==1:
            self.get_day()
        else:
            self.get_per_hour()

    def mult(self):
        p = Pool(6)
        for i in self.crawl_list:
            p.apply_async(self.crawl_save, args=(i,))
        print('Waiting for all subprocess done ...')
        p.close()
        p.join()
        print('All subprocess done')


In [33]:
befor_data = pd.read_csv('空气质量-taian_hour.csv',encoding='utf_8_sig')

In [45]:
last

3

In [49]:
last = len(befor_data)
befor_data.loc[last-1,'time']

'2019-04-30  20:00'

In [23]:
import pandas as pd
from lxml import etree
import time
import requests
import re
class get_data_hour:
headers = {'user_agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
url = 'http://www.tianqihoubao.com/aqi/qinhuangdao.html'
html = requests.get(url,headers=headers)
content = html.text
selector = etree.HTML(content)
time_ = selector.xpath('//div[@class="desc"]/text()')[0]
time_ =re.findall(r'数据更新时间:(.+)\(',time_)[0].strip()
data =pd.read_html(url, header=0, encoding='gbk')[0] 
data['time'] = time_
data.reindex(columns=[ 'time','监测点', 'AQI指数', '空气质量状况', 'PM10', 'Co', 'No2', 'So2', 'O3'])
data.set_index('time')

,监测点,AQI指数,空气质量状况,PM10,Co,No2,So2,O3
time,,,,,,,,
2019-04-30 19:00,北戴河环保局,61,良,72,0.5,16,6,119
2019-04-30 19:00,第一关,47,优,47,0.8,27,10,77
2019-04-30 19:00,监测站,53,良,56,0.7,69,8,67
2019-04-30 19:00,建设大厦,59,良,68,0.6,35,7,88
2019-04-30 19:00,文明里(启用171012),61,良,71,0.7,87,7,49


In [20]:
col = list(data.columns)

In [21]:
col

['监测点', 'AQI指数', '空气质量状况', 'PM10', 'Co', 'No2', 'So2', 'O3', 'time']

In [ ]:
|